In [2]:
%load_ext Cython


In [74]:
pyfhel = Pyfhel()

In [78]:
import Pyfhel

In [73]:
from Pyfhel import Pyfhel
from PyCtxt import PyCtxt
from PyPtxt import PyPtxt

In [75]:
import unittest
from Pyfhel import Pyfhel
from PyCtxt import PyCtxt
from PyPtxt import PyPtxt

class PyfhelTestCase(unittest.TestCase):
    def setUp(self):
        self.pyfhel = Pyfhel()

    def test_context_gen(self):
        self.pyfhel.ContextGen(65537)

    def test_key_gen(self):
        self.pyfhel.KeyGen()    
        
    def test_key_gen(self):
        self.assertRaises(ValueError, self.pyfhel.relinKeyGen(61))
        self.assertRaises(ValueError, self.pyfhel.relinKeyGen(0))
        self.pyfhel.relinKeyGen(60)     
        
    def test_widget_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')
        
if __name__ == '__main__':
    unittest.main()

In [76]:
a = PyfhelTestCase()

In [88]:
a.pyfhel.ContextGen(65537)
a.pyfhel.KeyGen()
a.assertRaises(ValueError, a.pyfhel.relinKeyGen(0))

SystemError: <built-in method relinKeyGen of Pyfhel.Pyfhel object at 0x7f6ada02cf50> returned a result with an error set

In [56]:
%%cython -c=-std=c++17 -lseal -lafhel
# distutils: language = c++

# Both numpy and the Cython declarations for numpy
import numpy as np

# Type checking for only numeric values
from numbers import Number

# Dereferencing pointers in Cython in a secure way
from cython.operator cimport dereference as deref

# Define Plaintext types
PLAINTEXT_T = (PyPtxt, float, int, np.ndarray)

cdef class Pyfhel:
    cdef Afseal* afseal           # The C++ methods are accessed via a pointer
    
    # INIT & DESTRUCT
    def __cinit__(self):
        self.afseal = new Afseal()
    def __dealloc__(self):
        if self.afseal != NULL:
            del self.afseal

    
    # ----------------------------- CRYPTOGRAPHY ------------------------------
    cpdef void ContextGen(self, long p, long m=2048, bool flagBatching=False,
                  long base=2, long sec=128, int intDigits=64, int fracDigits = 32):
        """Generates Homomorphic Encryption context based on parameters.
        
        Args:
            p (long): Plaintext modulus. All operations are modulo p.
            m (long=2048): Coefficient modulus. Higher allows more encrypted operations.
                      In batch mode it is the number of integers per ciphertext.
            flagBatching (bool=false): Set to true to enable batching.
            base (long=2): Polynomial base. Affects size of plaintexts/ciphertexts. 
            sec (long=128): Security level equivalent in AES. Either 128 or 192.
            intDigits (int=64): when encrypting/encoding double, truncated positions
                      dedicated to integer part, out of 'm' positions
            fracDigits (int=32): when encrypting/encoding double, truncated positions
                      dedicated to fractional part, out of 'm' positions
                      
        Return:
            None
        """
        self.afseal.ContextGen(p, m, flagBatching, base, sec,intDigits, fracDigits)
        
        
    cpdef void KeyGen(self):
        """Generates a pair of Private/Public Keys.
        
        Based on the current context, initializes one public and one private key. 
        Args:
            None
                      
        Return:
            None
        """
        self.afseal.KeyGen()
    
    # ENCRYPTION encrypt a PyPtxt object into a PyCtxt object
    cpdef encryptInt(self, int64_t value, PyCtxt ctxt=None):
        """Encrypts a single value into a PyCtxt ciphertext.
        
        Encrypts a single value using the current private key, based on the current context.
        Plaintext must either be an integer (int64_t) or a decimal that will get
        truncated (float).
        If provided a ciphertext, encrypts the plaintext inside it. 
        
        Args:
            ptxt (int|float): plaintext to encrypt.
            ctxt (PyCtxt=None): Optional destination ciphertext.  
            
        Return:
            PyCtxt: the ciphertext containing the encrypted plaintext
            
        Raise:
            TypeError: if the plaintext doesn't have a valid type.
        """
        if (ctxt._ptr_ctxt == NULL):
            ctxt = PyCtxt()
        self.afseal.encrypt(value, deref(ctxt._ptr_ctxt))
        return ctxt
    
    cpdef encryptFrac(self, double value, PyCtxt ctxt=None):
        """Encrypts a single value into a PyCtxt ciphertext.
        
        Encrypts a single value using the current private key, based on the current context.
        Plaintext must either be an integer (int64_t) or a decimal that will get
        truncated (float).
        If provided a ciphertext, encrypts the plaintext inside it. 
        
        Args:
            ptxt (int|float): plaintext to encrypt.
            ctxt (PyCtxt=None): Optional destination ciphertext.  
            
        Return:
            PyCtxt: the ciphertext containing the encrypted plaintext
            
        Raise:
            TypeError: if the plaintext doesn't have a valid type.
        """
        if (ctxt._ptr_ctxt == NULL):
            ctxt = PyCtxt()
        self.afseal.encrypt(value, deref(ctxt._ptr_ctxt))
        return ctxt
    
    cpdef encryptBatch(self, vector[int64_t] vec, PyCtxt ctxt=None): 
        """Encrypts a 1D numpy vector of integers into a PyCtxt ciphertext.
        
        Encrypts a 1D numpy vector of integers using the current private key,
        based on the current context. Plaintext must be a 1D numpy vector of integers.
        Requires batch mode. The vector needs to be in 'contiguous' or 'c' mode.
        If provided a ciphertext, encrypts the plaintext inside it. 
        
        Args:
            ptxt (np.ndarray[int, ndim=1, mode="c"]): plaintext to encrypt.
            ctxt (PyCtxt=None): Optional destination ciphertext.  
            
        Return:
            PyCtxt: the ciphertext containing the encrypted plaintext
        """
        if (ctxt._ptr_ctxt == NULL):
            ctxt = PyCtxt()
        self.afseal.encrypt(vec, deref(ctxt._ptr_ctxt)) 
        return ctxt  
        
    cpdef encryptPtxt(self, PyPtxt ptxt, PyCtxt ctxt=None):
        """Encrypts an already encoded PyPtxt plaintext into a PyCtxt ciphertext.
        
        Encrypts an already encoded PyPtxt plaintext using the current private key,
        based on the current context. Plaintext must be a PyPtxt.
        If provided a ciphertext, encrypts the plaintext inside it. 
        
        Args:
            ptxt (PyPtxt): plaintext to encrypt.
            ctxt (PyCtxt=None): Optional destination ciphertext.  
            
        Return:
            PyCtxt: the ciphertext containing the encrypted plaintext
            
        Raise:
            TypeError: if the plaintext doesn't have a valid type.
        """
        if (ptxt._ptr_ptxt == NULL):
            raise TypeError("Plaintext is empty")
        if (ctxt._ptr_ctxt == NULL):
            ctxt = PyCtxt()
        self.afseal.encrypt(deref(ptxt._ptr_ptxt), deref(ctxt._ptr_ctxt)) 
        ctxt.
        return ctxt

    def encrypt(self, ptxt not None, ctxt=None):
        """Encrypts any valid plaintext into a PyCtxt ciphertext.
        
        Encrypts a plaintext using the current private key, based on the current context.
        Plaintext must be an integer (int), a decimal that will get truncated (double),
        a PyPtxt encoded plaintext, or in Batch mode a 1D numpy vector of integers.
        Selects the encryption function based on type.
        
        If provided a ciphertext, encrypts the plaintext inside it. 
        
        Args:
            ptxt (PyPtxt|int|double|np_1d_int_array): plaintext to encrypt.
            ctxt (PyCtxt=None): Optional destination ciphertext.  
            
        Return:
            PyCtxt: the ciphertext containing the encrypted plaintext
            
        Raise:
            TypeError: if the plaintext doesn't have a valid type.
        """
        if not isinstance(ptxt, PLAINTEXT_T):
            raise TypeError('<Pyfhel ERROR> Plaintext type \['+type(ptxt)+
                            '\] not supported for encryption')
        if isinstance(ptxt, PyPtxt):
            return self.encryptPtxt(ptxt, ctxt)
        elif isinstance(ptxt, np.ndarray):
            if (ptxt.ndim is not 1) or (ptxt.dtype is not int):
                raise TypeError('<Pyfhel ERROR> Plaintext numpy array is not'
                                '1D vector of int values, cannot encrypt.')
            return self.encryptBatch(ptxt, ctxt)  
        elif isinstance(ptxt, float):
            return self.encryptValue(<float>ptxt, ctxt)   
        elif isinstance(ptxt, Number):
            return self.encryptValue(<int64_t>ptxt, ctxt)  
    
    cpdef decrypt(self, PyCtxt ctxt, PyPtxt ptxt=None):
        pass

In [ ]:

    # DECRYPT a PyCtxt object into a List of values
    def decrypt(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel decrypt error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))
        retList = []
        cdef vector[long] retVect
        ids = ctxt.getIDs()
        lens = ctxt.getLen()
        for i,l in zip(ids,lens):           # For each key in PyCtxt
            retPtxt = []
            retV = self.afseal.decrypt(i)    # Decrypt its Afhel Ctxt
            for k in range(l):
                retPtxt.append(retV[k])     # Append values 1 by 1 to final list
            retList.append(retPtxt)
        return retList                      # Return only the non-filler values


    # DUPLICATE a PyCtxt with all its parameters, useful to keep originals in ops
    def duplicate(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel set error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))
        ids = ctxt.getIDs()                 # Use same Pyfhel class and same #IDs
        new_ctxt = PyCtxt(ctxt.getPyfhel(), ctxt.getLen())
        for i in ids:                       # Loop over all IDs and append them
            new_ctxt.appendID(self.afseal.set(i))
        return new_ctxt                     # Return duplicated PyCtxt


    # ----------------------------- OPERATIONS --------------------------------
    # ADD two PyCtxt objects for each ID in both
    def add(self, ctxt1, ctxt2, neg=False):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel addCtxt error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel addCtxt error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))

        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        n_ids = len(ids1)
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::add to + each pair of Ctxts by IDs
            self.afseal.add(ids1[i],ids2[i], neg)


    # MULTiply two PyCtxt objects for each ID in both
    def mult(self, ctxt1, ctxt2):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))
        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        n_ids = len(ids1)
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::mult to * each pair of Ctxts by IDs
            self.afseal.mult(ids1[i],ids2[i])


    # MULTIPLY 3 PyCtxt objects for each ID in both
    def mult3(self, ctxt1, ctxt2, ctxt3):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))
        if not isinstance(ctxt3, PyCtxt):
            raise TypeError("Pyfhel multiplyBy error: ctxt3 must be of type PyCtxt instead of type " + str(type(ctxt3)))    
        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        ids3 = ctxt3.getIDs()
        n_ids = len(ids1)
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        if n_ids != len(ids3):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::mult to * each pair of Ctxts by IDs
            self.afseal.mult3(ids1[i],ids2[i], ids3[i])




    # SCALAR PRODuct between two PyCtxt objects for each ID in both
    def scalarProd(self, ctxt1, ctxt2):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel scalarProd error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel scalarProd error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))
        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        n_ids = len(ids1)
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::scalarProd to compute each pair of Ctxts by IDs
            self.afseal.scalarProd(ids1[i],ids2[i], 0)
            

    # SQUARE each cyphertext inside PyCtxt ctxt for each ID in it
    def square(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel square error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.square(ids[i])




    # CUMSUM Cumulative sum over all the values in the cyphertext
    def cumSum(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel cube error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.cumSum(ids[i])




    # CUBE each cyphertext inside PyCtxt ctxt for each ID in it
    def cube(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel cube error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.cube(ids[i])


    # NEGATE each cyphertext inside PyCtxt ctxt for each ID in it
    def negate(self, ctxt):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel negate error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.negate(ids[i])


    # COMPARE two PyCtxt objects for each ID in both
    def equalsTo(self, ctxt1, ctxt2):
        if not isinstance(ctxt1, PyCtxt):
            raise TypeError("Pyfhel equalsTo error: ctxt1 must be of type PyCtxt instead of type " + str(type(ctxt1)))
        if not isinstance(ctxt2, PyCtxt):
            raise TypeError("Pyfhel equalsTo error: ctxt2 must be of type PyCtxt instead of type " + str(type(ctxt2)))
        ids1 = ctxt1.getIDs()
        ids2 = ctxt2.getIDs()
        n_ids = len(ids1)
        comparison = []
        if n_ids != len(ids2):              # They must have the same # of IDs
            raise PyCtxtLenError()
        for i in range(n_ids):              # Use Afhel::equalsTo to * each pair of Ctxts by IDs
            comparison.append(self.afseal.mult(ids1[i],ids2[i]))
        return comparison


    # ROTATE each cyphertext inside PyCtxt ctxt for each ID in it
    def rotate(self, ctxt, c):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel rotate error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.rotate(ids[i], c)


    # SHIFT each cyphertext inside PyCtxt ctxt for each ID in it
    def shift(self, ctxt, c):
        if not isinstance(ctxt, PyCtxt):
            raise TypeError("Pyfhel rotate error: ctxt must be of type PyCtxt instead of type " + str(type(ctxt)))

        ids = ctxt.getIDs()
        n_ids = len(ids)

        for i in range(n_ids):
            self.afseal.shift(ids[i], c)



    # ----------------------------------- I/O ---------------------------------

    # SAVE ENVIRONMENT
    # Saves the environment into a .aenv file
    def saveEnv(self, fileName):
        if not isinstance(fileName, str):
            raise TypeError("Pyfhel saveEnv error: fileName must be of type str instead of type " + str(type(fileName)))
        self.afseal.saveEnv(fileName)
        return 


    # RESTORE ENVIRONMENT
    # Restores the environment from a .aenv file
    def restoreEnv(self, fileName):
        if not isinstance(fileName, str):
            raise TypeError("Pyfhel saveEnv error: fileName must be of type str instead of type " + str(type(fileName)))
        self.afseal.restoreEnv(fileName)
        self.modulus = long(pow(self.afseal.getP(), self.afseal.getR()))
        return

    #--------------------------------- AUXILIARY ------------------------------
    def numSlots(self):
        return self.afseal.numSlots()
    def getModulus(self):
        return self.modulus
    def delete(self, ctxt):
        for i in ctxt.getIDs():
            self.afseal.erase(i)



EXAMPLE ON USAGE OF STD STREAMS FROM C++

In [16]:
%%cython
# distutils: language = c++
from iostream cimport *    

def test_ofstreamss(bytes s):
    cdef ofstream* outputter
    # use try ... finally to ensure destructor is called
    outputter = new ofstream("output.txt")
    try:
        outputter.write(s,len(s))
    finally:
        del outputter
        
def test_ifstreamss(int n):
    cdef ifstream* inputer
    inputer = new ifstream("output.txt")
    s = b'aaaaaa'
    try:
        inputer.read(s,n)
    finally:
        del inputer
    return s

In [17]:
test_ofstreamss(b"This is a test\n")

In [18]:
test_ifstreamss(6)

b'This i'